In [2]:
from nltk import word_tokenize
from nltk import wordnet as wn
import re
import pandas as pd
import os
import nltk
from bs4 import BeautifulSoup
import string

In [3]:
# Import Personal Text Analysis Modules

os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Modules')
Cwd = os.getcwd()

import ccirelli2_general_text_analysis_module as cc_txt_analysis


In [4]:
# Import Excel Doc with Industry Keys and Key Words

os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Projects/Web Scraping Project - EEOC Articles')
Url = r'EEOC Article Study - Industry Values.xlsx'
df1 = pd.read_excel(Url)
df2 = df1.set_index('Keys') # Set the Major Industry Group as the index. 
df3 = df2.iloc[0:22, :]     # Limit the scope of the dataframe.  Function is picking up nan values which is causing
                            # an error. 

In [ ]:
# Import Articles File Names

In [5]:
chdir = os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Projects/Web Scraping project - EEOC Articles/EEOC Articles/')
Cdw = os.getcwd()
File_name_list = os.listdir()

In [ ]:
# Create Beautiful Soup Object of Each Article Title

In [6]:
def get_bs4Obj_of_Title_from_Texts(File_name_list):
        bs4_obj_list = []
        
        # Get File Direction 
        for file_name in File_name_list:
            if 'txt' in file_name:
                Dir = Cdw + '/' + file_name  
                File = open(Dir, 'rb')
                Text = File.read()
                bs4obj = BeautifulSoup(Text, 'html.parser')
                h1 = bs4obj.find('h1')
                if h1 == None:
                    bs4_obj_list.append(None)
                else:
                    bs4_obj_list.append(str(h1))
            
        return bs4_obj_list
    
bs4Obj_List_Titles = get_bs4Obj_of_Title_from_Texts(File_name_list)

In [ ]:
# Create a Diction of Punctuation Symbols

In [22]:
def create_punct_dict(list):
    Dict = {}
    for x in list:
        Dict[x] = ''
    return Dict

punct_dict = create_punct_dict(string.punctuation)

In [ ]:
# Tokenize and Clean Text

In [119]:
def tokenize_title_clean_bs4Objs_returnListObjs(bs4Obj_list):
        
    # Define list to catch tokenized object
    bs4Obj_tokenized_list = []
    
    # Tokenize each object
    for bs4Obj in bs4Obj_list:
        bs4Obj_string = str(bs4Obj)
        bs4Obj_tokenized = nltk.wordpunct_tokenize(bs4Obj_string)
        bs4Obj_is_alpha = cc_txt_analysis.get_isalpha(bs4Obj_tokenized)
        bs4Obj_tokenized_lower = [x.lower() for x in bs4Obj_is_alpha]
        bs4Obj_tokenized_list.append(bs4Obj_tokenized_lower)
    
    # Return bs4Obj List of tokenized objects
    
    return bs4Obj_tokenized_list

Tokenized_title_list = tokenize_title_clean_bs4Objs_returnListObjs(bs4Obj_List_Titles[:1])
    

In [ ]:
# Create Tokenized version of Text Body - Put Into a List

In [118]:
def tokenize_textBody_cleanText_returnListObjs(File_name_list):
        Clean_text_list = []
        
        for file_name in File_name_list:
            if 'txt' in file_name:
                Dir = Cdw + '/' + file_name  
                File = open(Dir, 'rb')
                Text = File.read()
                Text_str = str(Text)
                Clean_text = cc_txt_analysis.text_cleaning_pipeline_Single_Text_pre_tokenize(Text_str)
                Clean_text_list.append(Clean_text)
        
        return Clean_text_list

Tokenized_textBody_list = tokenize_textBody_cleanText_returnListObjs(File_name_list[:1])

In [ ]:
# Create Industry Dictionary of Key Word Synonyms

In [23]:
def Create_Dictionary_Industry_KeyWord_Synonyms():
    os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Projects/Web Scraping Project - EEOC Articles')
    Url = r'EEOC Article Study - Industry Values.xlsx'
    df1 = pd.read_excel(Url)
    df2 = df1.set_index('Keys')  
    dataframe = df2.iloc[0:22, :]      #should have just recreated this list in Python. 
                             
    Dict_major = {}
    
    # for each major industry value (column A) in the dataframe
    for major in dataframe.index:
        
        # Define the minor industry as the row values. 
        df_minor = dataframe.loc[major]
        
        # Define the industry value lvl 2 and 3 to include in the keys of your dictionary (for descriptive purp)
        Industry_lvl_2 = df_minor['Division']
        Industry_lvl_3 = df_minor['Column 1']

        # Define the list to catch the synsets generated by the below function. 
        Synset_list = []
                
        # For the value in the dataframe, the key being the value in column B 'Division'
        for value in df_minor[1:]:
            # Verify that it is an instance of a string as we have None values in the dataframe. 
            if isinstance(value, str):
                # Convert the value to lowercase. 
                value_lower = value.lower()
                # Generate synsets for this value.
                Synset = wn.wordnet.synsets(value_lower)
                # Extract the word from the synset object ('word.n.01')
                Lemma_names = [x.lemma_names() for x in Synset]
                # Lemman_names is a list of lists.  Iterate over each list. 
                for List in Lemma_names:
                    # Get the words in each sub list. 
                    for word in List:
                        # We want to end up with a Set of unique values.  Therefore, check to see 
                        # if the word is already in our list. 
                        if word not in Synset_list:
                            # If not, then append the word to our list. 
                            Synset_list.append(word)
        
        # Create the name of the Major Industry Group that will constitute the Keys of our Dict. 
        Industry_identifier = (str(major) + ' ' + Industry_lvl_2 + ' - ' + Industry_lvl_3)
        # Join the Keys with our matching values. 
        Dict_major[Industry_identifier] = Synset_list
    
    # Return our completed Industry Dictionary
    return Dict_major

In [24]:
Industry_Dictionary = Create_Dictionary_Industry_KeyWord_Synonyms()

In [ ]:
# Create Industry Classifier - Compares Text to Industry Dict & Counts Matches

In [149]:
def Get_Text_Word_Industry_Count(Tokenized_title_list, Tokenized_textBody_list, Dict_Industries):

    Word_match_count = {}
    
    for title, text in zip(Tokenized_title_list, Tokenized_textBody_list): 
                    
            for title_token, body_token in zip(title, text):       
            
                # for each industry category
                for industry_category in Dict_Industries:     
        
                    # if the title token is found within the industry category values
                    if title_token in Dict_Industries[industry_category]:
                
                        # if it is, check to see if the industry category is already in the Word_match_dict 
                        if industry_category in Word_match_count.keys():
 
                        # If the industry category is already in the dict, check to see if the title_token has already
                        # been added to this key. 
                            if title_token in Word_match_count[industry_category]:
                         
                                # if the value already exists for this key, then just add another title token 
                                Word_match_count[industry_category] = Word_match_count[industry_category] + [title_token]
                        
                            # if the title token is not present, but we have the key, no prob, just add a token.  
                            else: 
                                Word_match_count[industry_category] = [title_token]
                    
                        # if the industry category is not already in the keys, but we have a match, add key and dict
                        else:
                            Word_match_count[industry_category] = [title_token]
                
                    # Now, if the title token is not in the Dictionary, we dont have a match, so lets see if we can 
                    # match a token in the text body to our Industry Dictionary. 
                
                    else:
                        if body_token in Dict_Industries[industry_category]:
                            if industry_category in Word_match_count.keys():
                                if body_token in Word_match_count[industry_category]:
                                    Word_match_count[industry_category] = Word_match_count[industry_category] + [body_token]
                                else:
                                    Word_match_count[industry_category] = [body_token]
                            else:
                                Word_match_count[industry_category] = [body_token]
                        # if body_token not in Dict_values, then neither our title_tokens nor body_tokens found 
                        # a match. 
                        
    for x in Word_match_count:
        Word_match_count[x] = len(Word_match_count[x])            
    
    df = pd.DataFrame(Word_match_count, index = [1])
    df_tran = pd.DataFrame.transpose(df)
    df_sorted = df_tran.sort_values(1, ascending = False)
    
    return df_sorted

### This is an issue.  You dont know if your two lists match in terms of the order of the articles. 
### You should have embedded within one function 

In [150]:
Test = Get_Text_Word_Industry_Count(Tokenized_title_list,
                                    Tokenized_textBody_list, 
                                    Industry_Dictionary)


print(Test)

# You're missing a loop.  You need to loop over the List of tokenized text, then loop over the tokens. 

                                             1
Major Group 73 Services - Business Services  1


In [35]:
# Get Industry Prediction - Take the top row of the dataframe as the highest num matches as our prediction. 

In [151]:
def Industry_prediction(Obj_from_industry_classifier):
    # Return the top row as the Prediction from our Industry Classifier function. 
    Top_row = Obj_from_industry_classifier.iloc[0]
    Industry = Top_row.name
    return Industry

In [152]:
Industry_prediction(Test)

'Major Group 73 Services - Business Services'

In [40]:
# Careful, your Classifier is iterating over a list.  Maybe you should embed the prediction in the classifier.
# 

In [41]:
# Put It All Together

In [42]:
def predict_industry_pipeline(Text, Industry_dataframe):
    Text_tokenized = word_tokenize(Text)
    Clean_text = text_cleaning_pipeline(Text_tokenized)
    Industry_Dictionary = Create_Industry_Dictionary(Industry_dataframe)
    Industry_classifier = get_industry(Clean_text, Dict_Industries_dict)
    Industry_prediction = Industry_prediction(Industry_classifier)
    return Industry_prediction

In [43]:
# Loop over individual texts to get prediction

In [52]:
chdir = os.chdir(r'/Users/ccirelli2/Public/Python Programing Docs/Projects/Web Scraping project - EEOC Articles/EEOC Articles/')
Url_name_list = os.listdir()

for x in Url_name_list[:10]:
    if '.txt' in x:
        File = open(x, 'rb')
        Text_bytes = File.read()
        Text_string = str(Text_bytes)
        
        Text_tokenized = nltk.word_tokenize(Text_string)
        Text_strip_punct = cc.strip_punctuation(Text_tokenized)
        Text_strip_stop_words = cc.strip_stop_words(Text_strip_punct)
        Text_strip_forwardslash = cc.strip_tokens_forwardSlash_x2(Text_strip_stop_words)
        Text_strip_two_variable_tokens = cc.strip_two_variable_tokens(Text_strip_forwardslash)
        Text_get_isalpha = cc.get_isalpha(Text_strip_two_variable_tokens)
        Final_text = [x.lower() for x in Text_get_isalpha]

        
        Industry_Dictionary = Create_Industry_Dictionary(df3)
        Industry_count = Get_Text_Industry_Count(Final_text, Industry_Dictionary)
        print(Industry_prediction(Industry_count))
        
        #### Something is wrong with the Get_Text_Industry_Count Function.  It is returning empty dataframes. 
        


Major Group 87 Services - Accounting
Major Group 20-39 Manufacturing - Manufacture
Major Group 81 Services - Law firm
Major Group 87 Services - Accounting
Major Group 91-99 Services - Government
Major Group 50-51 Wholesale - Wholesale
Major Group 91-99 Services - Government
Major Group 60-67 Finance - Bank 
Major Group 81 Services - Law firm
Major Group 73 Services - Business Services
